In [1]:
from typing import Tuple
import os
import sys
import torch
import fire
import time
import json
import logging

from pathlib import Path

from fairscale.nn.model_parallel.initialize import initialize_model_parallel

from llama import ModelArgs, Transformer, Tokenizer, LLaMA


def setup_model_parallel() -> Tuple[int, int]:
    local_rank = int(os.environ.get("LOCAL_RANK", -1))
    world_size = int(os.environ.get("WORLD_SIZE", -1))

    torch.distributed.init_process_group("nccl")
    initialize_model_parallel(world_size)
    torch.cuda.set_device(local_rank)

    # seed must be the same in all processes
    torch.manual_seed(1)
    return local_rank, world_size


def load(
    ckpt_dir: str,
    tokenizer_path: str,
    local_rank: int,
    world_size: int,
    max_seq_len: int,
    max_batch_size: int,
) -> LLaMA:
    start_time = time.time()
    checkpoints = sorted(Path(ckpt_dir).glob("*.pth"))
    assert world_size == len(
        checkpoints
    ), f"Loading a checkpoint for MP={len(checkpoints)} but world size is {world_size}"
    ckpt_path = checkpoints[local_rank]
    logging.info("Loading")
    checkpoint = torch.load(ckpt_path, map_location="cpu")
    with open(Path(ckpt_dir) / "params.json", "r") as f:
        params = json.loads(f.read())

    model_args: ModelArgs = ModelArgs(
        max_seq_len=max_seq_len, max_batch_size=max_batch_size, **params
    )
    tokenizer = Tokenizer(model_path=tokenizer_path)
    model_args.vocab_size = tokenizer.n_words
    torch.set_default_tensor_type(torch.cuda.HalfTensor)
    model = Transformer(model_args)
    torch.set_default_tensor_type(torch.FloatTensor)
    model.load_state_dict(checkpoint, strict=False)

    generator = LLaMA(model, tokenizer)
    logging.info(f"Loaded in {time.time() - start_time:.2f} seconds")
    return generator

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ckpt_dir = "/netscratch/nehring/models/llama/7B/"
tokenizer_path = "/netscratch/nehring/models/llama/tokenizer.model"
temperature = 0.8
top_p = 0.95
max_seq_len = 512
max_batch_size = 32

local_rank, world_size = setup_model_parallel()
if local_rank > 0:
    sys.stdout = open(os.devnull, "w")

generator = load(
    ckpt_dir, tokenizer_path, local_rank, world_size, max_seq_len, max_batch_size
)

def generate(prompt):
    print(generator.generate(
        [prompt], max_gen_len=256, temperature=temperature, top_p=top_p
    )[0])
    print()


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loading
Loaded in 25.14 seconds


In [4]:
#generate("Dr. Sven Schmeier ist")
#generate("Dr. Sven Schmeier is")
#generate("2+5=")

generate("Folgende Versicherungsgesellschaften bieten in Deutschland eine Rechtschutzversicherung an:")

Folgende Versicherungsgesellschaften bieten in Deutschland eine Rechtschutzversicherung an:
Die Rechtsschutzversicherungen unterliegen der Unternehmenssteuer. Die Versicherungsunternehmen sind in der Rechtsdienstleisterdatei des Finanzministeriums zu finden.
Die Anspruchserhebung für die Rechtsschutzversicherung erfolgt nach dem Grundsatz der Rückzahlung (Zwischenabzug). Der Versicherungsbeitrag ist im KVR (Deutsche Kreditwirtschaft) mit 3,0 % unter der gesetzlichen Minderjährigkeitsgrenze (16 Jahre) versteuert.
Die Versicherung für Rechtsanwaltskosten wird durch die Anwaltskassensorgepflichten unterstützt. Die Versicherung für Sachkosten wird durch die Sachverständigenkostenversicherung des Bunds unterstützt.
Werden die versicherten Rechtsstreitigkeiten nicht innerhalb der vereinbarten Fristen abgeschlossen, haben die versicherten Personen das Recht, die zu erbringenden Rechtsausgaben in der vorgesehenen Höhe auf die entsprechende Versicherungsleistung zu verzichten.
Durch diese einhe

In [24]:
context = '''Wie teuer Döner schon geworden ist
Der Döner als schnelles Essen für zwischendurch ist beliebter als Currywurst. Auch hier sind die Preise stark gestiegen. Allerdings gibt es große Unterschiede zwischen den Bundesländern.
Von Lilli-Marie Hiltscher, hr
Döner gilt als schnelles, billiges Essen für zwischendurch - für ein paar Euro zu haben. Die flächendeckende Verfügbarkeit und die niedrigen Preise haben ihn zu einem der Lieblings-Fast-Food-Gerichte der Deutschen werden lassen. Doch die Zeiten des Drei-Euro-Döners sind vorbei. Die Gastronomen kämpfen mit der hohen Inflation und sind gezwungen, ihre Preise anzuheben.
Kostendeckend erst ab neun Euro? "Wenn wir den Döner kostendeckend verkaufen wollten, müssten wir die Preise auf mindestens neun Euro anheben", sagt Ömer Gülec. Er betreibt gemeinsam mit seinem Bruder einen Dönerladen in Frankfurt am Main und musste den Preis für seinen Döner im Februar bereits einmal anheben.Viele seiner Kunden könnten den dringend notwendigen Schritt hin zu höheren Preisen zwar verstehen. Aber: "Für manche Kunden ist der Döner ein Grundnahrungsmittel, das nicht mehr als sechs Euro kosten darf." Also müsse er gut kalkulieren, damit die Kunden nicht plötzlich wegen zu hoher Preise fernbleiben.
'''

prompt = context + "Der Döner ist kostendeckend ab"
generate(prompt)

Wie teuer Döner schon geworden ist
Der Döner als schnelles Essen für zwischendurch ist beliebter als Currywurst. Auch hier sind die Preise stark gestiegen. Allerdings gibt es große Unterschiede zwischen den Bundesländern.
Von Lilli-Marie Hiltscher, hr
Döner gilt als schnelles, billiges Essen für zwischendurch - für ein paar Euro zu haben. Die flächendeckende Verfügbarkeit und die niedrigen Preise haben ihn zu einem der Lieblings-Fast-Food-Gerichte der Deutschen werden lassen. Doch die Zeiten des Drei-Euro-Döners sind vorbei. Die Gastronomen kämpfen mit der hohen Inflation und sind gezwungen, ihre Preise anzuheben.
Kostendeckend erst ab neun Euro? "Wenn wir den Döner kostendeckend verkaufen wollten, müssten wir die Preise auf mindestens neun Euro anheben", sagt Ömer Gülec. Er betreibt gemeinsam mit seinem Bruder einen Dönerladen in Frankfurt am Main und musste den Preis für seinen Döner im Februar bereits einmal anheben.Viele seiner Kunden könnten den dringend notwendigen Schritt hin zu